In [1]:
!mkdir ade20k
!wget http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip -P ade20k

--2022-08-16 11:29:07--  http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 967382037 (923M) [application/zip]
Saving to: ‘ade20k/ADEChallengeData2016.zip’

ADEChallengeData201 100%[===================>] 922.57M  83.8MB/s    in 11s     

2022-08-16 11:29:18 (83.5 MB/s) - ‘ade20k/ADEChallengeData2016.zip’ saved [967382037/967382037]



In [2]:
#!wget http://data.csail.mit.edu/places/ADEchallenge/release_test.zip -P ade20k

In [3]:
!pip3 install openmim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 2.8 MB/s 
     |████████████████████████████████| 235 kB 10.3 MB/s 
     |████████████████████████████████| 51 kB 9.2 MB/s 


In [4]:
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.6 MB 9.0 MB/s 
     |████████████████████████████████| 190 kB 4.7 MB/s 


In [5]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 10481, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 10481 (delta 160), reused 290 (delta 130), pack-reused 10125
Receiving objects: 100% (10481/10481), 14.60 MiB | 29.14 MiB/s, done.
Resolving deltas: 100% (7590/7590), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 578 kB 4.7 MB/s 
  Running setup.py develop for mmsegmentation


In [6]:
!mkdir /content/mmsegmentation/data
!mkdir /content/mmsegmentation/data/ade

In [7]:
!unzip -q /content/ade20k/ADEChallengeData2016.zip -d /content/mmsegmentation/data/ade

In [8]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.1+cu113 True
0.27.0


In [9]:
!mkdir checkpoints

!wget https://download.openmmlab.com/mmsegmentation/v0.5/segmenter/segmenter_vit-b_mask_8x1_512x512_160k_ade20k/segmenter_vit-b_mask_8x1_512x512_160k_ade20k_20220105_151706-bc533b08.pth -P checkpoints

--2022-08-16 11:30:15--  https://download.openmmlab.com/mmsegmentation/v0.5/segmenter/segmenter_vit-b_mask_8x1_512x512_160k_ade20k/segmenter_vit-b_mask_8x1_512x512_160k_ade20k_20220105_151706-bc533b08.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.89.140.71
Connecting to download.openmmlab.com (download.openmmlab.com)|47.89.140.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410082270 (391M) [application/octet-stream]
Saving to: ‘checkpoints/segmenter_vit-b_mask_8x1_512x512_160k_ade20k_20220105_151706-bc533b08.pth’

segmenter_vit-b_mas 100%[===================>] 391.08M  8.83MB/s    in 45s     

2022-08-16 11:31:01 (8.66 MB/s) - ‘checkpoints/segmenter_vit-b_mask_8x1_512x512_160k_ade20k_20220105_151706-bc533b08.pth’ saved [410082270/410082270]



In [10]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [11]:

from mmcv import Config

config_file='/content/mmsegmentation/configs/segmenter/segmenter_vit-b_mask_8x1_512x512_160k_ade20k.py'

checkpoint_file = 'checkpoints/segmenter_vit-b_mask_8x1_512x512_160k_ade20k_20220105_151706-bc533b08.pth'

cfg = Config.fromfile(config_file)


In [12]:
'''
from mmseg.apis import set_random_seed
from mmseg.utils import get_device


# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

cfg.runner.max_iters = 200
cfg.log_config.interval = 10
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 200

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'



# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()

'''

"\nfrom mmseg.apis import set_random_seed\nfrom mmseg.utils import get_device\n\n\n# Since we use only one GPU, BN is used instead of SyncBN\ncfg.norm_cfg = dict(type='BN', requires_grad=True)\ncfg.model.backbone.norm_cfg = cfg.norm_cfg\ncfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg\n\ncfg.runner.max_iters = 200\ncfg.log_config.interval = 10\ncfg.evaluation.interval = 200\ncfg.checkpoint_config.interval = 200\n\n# We can still use the pre-trained Mask RCNN model though we do not need to\n# use the mask branch\ncfg.load_from = checkpoint_file\n\n# Set up working dir to save files and logs.\ncfg.work_dir = './work_dirs/tutorial'\n\n\n\n# Set seed to facitate reproducing the result\ncfg.seed = 0\nset_random_seed(0, deterministic=False)\ncfg.gpu_ids = range(1)\ncfg.device = get_device()\n\n"

In [13]:
cfg.runner.max_iters = 200
cfg.log_config.interval = 10
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 200

In [14]:
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

cfg.load_from = checkpoint_file
cfg.work_dir = './work_dirs/tutorial'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()


In [15]:
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
checkpoint = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segmenter/vit_base_p16_384_20220308-96dfe169.pth'
backbone_norm_cfg = dict(type='LN', eps=1e-06, requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=
    'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segmenter/vit_base_p16_384_20220308-96dfe169.pth',
    backbone=dict(
        type='VisionTransformer',
        img_size=(512, 512),
        patch_size=16,
        in_channels=3,
        embed_dims=768,
        num_layers=12,
        num_heads=12,
        drop_path_rate=0.1,
        attn_drop_rate=0.0,
        drop_rate=0.0,
        final_norm=True,
        norm_cfg=dict(type='LN', eps=1e-06, requires_grad=True),
        with_cls_token=True,
        interpolate_mode='bicubic'),
    decode_head=dict(
        type='SegmenterMaskTransformerHead',
        in_channels=768,
        channels=768,
        num_classes=150,
        num_layers=2,
        num_heads=12,
        embed

In [16]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv
import os.path as osp


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2022-08-16 11:31:03,239 - mmseg - INFO - Loaded 20210 images
/content/mmsegmentation/mmseg/models/backbones/vit.py:219: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower t

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 4.5 task/s, elapsed: 442s, ETA:     0s

2022-08-16 11:39:55,056 - mmseg - INFO - per class results:
2022-08-16 11:39:55,069 - mmseg - INFO - 
+---------------------+-------+-------+
|        Class        |  IoU  |  Acc  |
+---------------------+-------+-------+
|         wall        | 71.88 | 85.15 |
|       building      | 78.36 | 90.46 |
|         sky         | 92.47 | 97.21 |
|        floor        | 73.56 | 80.21 |
|         tree        | 70.96 | 83.03 |
|       ceiling       | 81.22 | 91.13 |
|         road        | 59.12 | 64.23 |
|         bed         | 79.22 | 96.36 |
|      windowpane     |  54.8 | 85.25 |
|        grass        | 41.18 | 54.56 |
|       cabinet       | 52.99 | 64.57 |
|       sidewalk      | 42.98 | 72.48 |
|        person       | 72.22 | 78.66 |
|        earth        | 14.28 | 16.79 |
|         door        | 38.11 | 48.59 |
|        table        |  47.9 | 67.06 |
|       mountain      | 57.71 | 72.03 |
|        plant        | 49.75 | 67.77 |
|       curtain       | 67.93 | 79.08 |
|        chair    

In [21]:
import matplotlib.pyplot as plt
from mmseg.core.evaluation import get_palette
model.cfg = cfg
#img = '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/validation/ADE_val_00001000.jpg'
resimler=['validation/ADE_val_00001962.jpg' , 'validation/ADE_val_00001963.jpg' , 'validation/ADE_val_00001457.jpg' , 
          'validation/ADE_val_00001458.jpg' , 'validation/ADE_val_00001459.jpg' , 'training/ADE_train_00000565.jpg' ,
          'training/ADE_train_00000936.jpg' , 'training/ADE_train_00000952.jpg' , 'training/ADE_train_00003194.jpg' , 
          'training/ADE_train_00006845.jpg']
for img in resimler:
  result = inference_segmentor(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img)
  show_result_pyplot(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img, result,get_palette('ade20k'))

Output hidden; open in https://colab.research.google.com to view.